NOTE : The execution of sampling took too much time, and I did not completely ran this notebook.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from cmdstanpy import CmdStanModel
from matplotlib import pyplot as plt

# Load and inspect data

In [ ]:
df = pd.read_csv("data/data-attendance-3.txt")
print(len(df))
print(df.dtypes)
df.head(5)

# Run Stan

In [ ]:
model = CmdStanModel(stan_file="model/model5-5-exec4.stan")
print(model)

In [ ]:
data = {
    "I" : len(df),
    "A" : df["A"].values,
    "Score" : df["Score"].values/200,
    "Cloudy" : [1 if x == "B" else 0 for x in df["Weather"].values],
    "Rainy" : [1 if x == "C" else 0 for x in df["Weather"].values],
    "Y" : df["Y"].values
}

In [ ]:
fit = model.sample(data=data)

# Inspect the result

In [ ]:
print(type(fit))
print(fit)

In [ ]:
print(fit.num_draws)
print(fit.draws().shape)

In [ ]:
fit.stan_variables()

In [ ]:
fit_summary = fit.summary()
fit_summary

In [ ]:
# Check the convergence
fit_summary[fit_summary["R_hat"] >= 1.1]

In [ ]:
print(fit.diagnose())

# Plot the result

In [ ]:
q_percentiles = np.percentile(fit.stan_variable("q"), [10.0, 50.0, 90.0], axis=0)
q_percentiles.shape

In [ ]:
# Plot score vs q (for weather = 'A' and 'A' = 0 only)

df_tmp = df.sort_values(by=["Score"])[["Weather", "A"]]
idx = df_tmp[(df_tmp["Weather"] == 'A') & (df_tmp['A'] == 0)].index.values

fig = plt.figure(figsize=(8, 6))
ax = plt.subplot(111)

ax.plot(df["Score"][idx], df["Y"][idx] + 0.1*np.random.randn(len(idx)), 'o', markersize=5, color="C0")
ax.plot(df["Score"][idx], q_percentiles[1][idx], '.-', color="C1")
ax.fill_between(df["Score"][idx], q_percentiles[0][idx], q_percentiles[2][idx], alpha=0.3, color="C1")
ax.set_xlabel("Score")
ax.set_ylabel("q")

In [ ]:
# prob vs. observation

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

ax = axes[0]
sns.violinplot(x=q_percentiles[1], y=df["Y"], color='white', linewidth=5, orient='h', ax=ax)
ax.set_xlabel("q(median)")

ax = axes[1]
sns.violinplot(x=q_percentiles[1], y=df["Y"], color='white', linewidth=5, orient='h', ax=ax)
sns.stripplot(x=q_percentiles[1], y=df["Y"], hue=df["A"], orient='h', ax=ax)
ax.set_xlabel("q(median)")

plt.show()

In [ ]:
# ROC curve : 

print("ROC AUC Score :", roc_auc_score(df["Y"].values, q_percentiles[1]))

fpr, tpr, thresholds = roc_curve(df["Y"].values, q_percentiles[1])
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
ax.plot(fpr, tpr)
ax.plot([0,1], [0,1], 'k--')
ax.set_aspect("equal")
ax.set_xlabel("False Positive rate")
ax.set_xlabel("True Positive rate")
plt.show()